# Elasticsearch Recipe Indexer

This Python script indexes your recipe `.txt` files into **Elasticsearch** using the **Italian analyzer**,  
allowing full-text search across recipe titles and contents.

It automatically:
- Creates or resets the index (`index_recipes`)
- Bulk-indexes all recipes from the `files/` folder
- Supports both `match` and `match_phrase` queries

In [ ]:
from elasticsearch import Elasticsearch, helpers
import time
import os

# Connect to Elasticsearch
ES_HOST = "http://localhost:9200"
es = Elasticsearch([ES_HOST])

# Delete the index if it exists
if es.indices.exists(index='index_recipes'):
    es.indices.delete(index='index_recipes')

# Define the mapping for the index
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text",
                      "analyzer": "italian",
                      "search_analyzer": "italian"},
            "content": {"type": "text", 
                        "analyzer": "italian",
                        "search_analyzer": "italian"}
        }
    }
}

def bulk_index(directory, index_name):
    actions = []

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            path = os.path.join(directory, filename)
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()

            # Remove file extension for title
            title = os.path.splitext(filename)[0]
            
            action = {
                "_index": index_name,
                "_source": {
                    "title": title,
                    "content": text
                }
            }
            actions.append(action)

    # Bulk index the documents with only one call to Elasticsearch
    if actions:
        helpers.bulk(es, actions)
        print(f"{len(actions)} documents indexed in the '{index_name}' index.")
    else:
        print("No .txt files found in the directory.")

def search(query):
   res = es.search(index='index_recipes', body=query)
   print("Search Results:")
   for i, hit in enumerate(res['hits']['hits']):
    res_doc = hit['_source']
    print(f"{i+1}. Title: {res_doc['title']}\n   Content: {res_doc['content']}\n")

## Measure Indexing Time

This snippet measures how long the indexing process takes.

In [6]:
# Index documents from the 'files' directory
before = time.time()
es.indices.create(index='index_recipes', body=mapping)
bulk_index("files", "index_recipes")
after = time.time()
print(f"Indexing took {after - before} seconds")

5939 documents indexed in the 'index_recipes' index.
Indexing took 5.105345964431763 seconds


## Interactive Search Query (with unlimited results)

This snippet lets the user enter a search query in natural form and builds  
an Elasticsearch query dynamically, supporting both `match` and `match_phrase`.

In [11]:
# Ask the user to insert a query
print("\nEnter your search query using one of the following formats:")
print('  • title: tiramisu')
print('  • content: "burro e salvia"')
print('  • content: banane')
print("Use quotes for exact phrase search.\n")

query = input("Search → ").strip()

# Validate the input
if ":" not in query:
    print("Invalid format. Please use 'title:' or 'content:' before your search term.")
else:
    type_query, content_query = query.split(":", 1)
    type_query = type_query.strip().lower()
    content_query = content_query.strip()

    # Build the query body
    if type_query in ["title", "content"]:
        if content_query.startswith('"') and content_query.endswith('"'):
            phrase = content_query.strip('"')
            body = {"query": {"match_phrase": {type_query: phrase}}, "size": 10000}
        else:
            body = {"query": {"match": {type_query: content_query}}, "size": 10000}
        search(body)  # Call your search function
    else:
        print("Unknown field. Use either 'title:' or 'content:'.")



Enter your search query using one of the following formats:
  • title: tiramisu
  • content: "burro e salvia"
  • content: banane
Use quotes for exact phrase search.

Search Results:
1. Title: Tiramisù
   Content: Per preparare il tiramisù preparate il caffé con la moka per ottenerne 300 g, poi zuccherate a piacere (noi abbiamo messo un cucchiaino) e lasciatelo raffreddare in una ciotolina bassa e ampia. Separate le uova dividendo gli albumi dai tuorli ricordando che per montare bene gli albumi non dovranno presentare alcuna traccia di tuorlo. Montate i tuorli con le fruste elettriche, versando solo metà dose di zucchero Non appena il composto sarà diventato chiaro e spumoso, e con le fruste ancora in funzione, potrete aggiungere il mascarpone, poco alla volta Incorporato tutto il formaggio avrete ottenuto una crema densa e compatta Quando saranno schiumosi versate il restante zucchero un po’ alla volta Dovrete montarli a neve ben ferma così stempererete il composto. Dopodiché procede